# 4.2.4 [バッチ対応版] 交差エントロピー誤差の実装

In [1]:
import numpy as np

In [7]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size


In [8]:
# 「２」を正解とする
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [9]:
# 例１：「２」の確率が最も高い場合（０．６）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

# この結果はミニバッチでないときと同じ

0.51082545709933802

In [10]:
# 教師データtがone-hot表現でなくラベルとして与えられた時の交差エントロピー誤差
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [11]:
# 「２」を正解とする
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [12]:
# 例１：「２」の確率が最も高い場合（０．６）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

# この結果は、正解をラベルでなくone-hotで与えているので、当然、間違い。

23.718987110506905

In [13]:
# 「２」を正解とするのを、ラベルで表現する(のを、試してみた。)
t = [2]

# 例１：「２」の確率が最も高い場合（０．６）
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))



0.51082545709933802

In [15]:
# この結果は、合っている!!
# なぜだ ｔとyの次元数があってなくても良いのか(あってなくても良いように作ってあるということか?)

- こう書いてある
  * 実装のポイントは、one-hot表現でtが0の要素は、交差エントロピー誤差も0であるから、その計算は無視しても良いということです。
  * 言い換えれば、正解ラベルに対してニューラルネットワークの出力を得ることができれば、交差エントロピー誤差を計算することができるのです。

- ひとつずつ、確認する。


1. one-hot表現でtが0の要素
 - one-hot表現とは
  - 正解ラベルを１として、それ以外は０で表す表記法
2. 「正解ラベルを１として、それ以外は０で表すという表記」で、tが０の要素
3. すなわち「不正解の要素」ということだ。

つまり

「実装のポイントは、one-hot表現でtが0の要素は、交差エントロピー誤差も0であるから、その計算は無視しても良いということです。」


ここで言いたいことは、こういうことではないだろうか。

- 実装のポイントは、不正解の要素は、誤差もゼロであるから、無視して良いということ。

- 不正解とわかった時点で、以降の計算もしなくていいはずだ、ということ。


言い換えれば

- 正解ラベルに対してニューラルネットワークの出力を得ることができる  
  - = 誤差があることがわかる
    - = 誤差を計算する必要がある

本書ではこういう言い方はしてないようだが、自分はこのように理解した。


In [ ]:
# np.log(y[np.arange(batch_size),t])

In [24]:
# np.arange(batch_size) => 0からbatch_size-1までの配列を生成する
np.arange(5)

array([0, 1, 2, 3, 4])

In [ ]:
t = [2, 7, 0, 9, 4]
# なら
# np.log(y[np.arange(batch_size),t])
# = np.log(  [ y[0,2],y[1,7],y[2,0]... ]  )

numpyに、arrangeを使って、配列を生成させる、というところが

「ミニバッチ」ということなのだろう。